In [1]:
from pathlib import Path
import os

os.chdir(Path(os.getcwd()).parent.parent)

In [2]:
from os.path import join

import json
import ast
import timeit

import pandas as pd
import numpy as np
from elasticsearch import AsyncElasticsearch
from collections import Counter

In [3]:
root_path = os.getcwd()
data_path = join(os.getcwd(), "data")
raw_path = join(data_path, "raw")
pre_path = join(data_path, "preprocessing")
bulk_path = join(data_path, "bulk")

## Load dictionary_i.csv

In [4]:
dictionary_df = pd.DataFrame(columns=["dictionary"])

In [5]:
for i in range(2):
    print(f"i: {i}")
    df = pd.read_csv(join(pre_path, f"dictionary_{i}.csv"))
    dictionary_df = pd.concat([dictionary_df, df], ignore_index=True)
    print(dictionary_df.shape)

i: 0
(20430, 1)
i: 1
(92144, 1)


In [6]:
dict_series = dictionary_df["dictionary"]
dict_series = dict_series.drop_duplicates(ignore_index=True)
dict_series = dict_series.sort_values(ignore_index=True)
len(dict_series)

72091

In [7]:
new_dict_df = pd.DataFrame(data=dict_series, columns=["dictionary"])
new_dict_df.shape

(72091, 1)

In [8]:
new_dict_df.head()

,dictionary
0,read
1,ㄱ
2,ㄴ
3,ㄷ
4,ㄹ


In [9]:
new_dict_df.to_csv(join(pre_path, f"dictionary.csv"), index=False, encoding="utf-8-sig")

## Bulk

In [14]:
action_str = {
    "index": {
        "_index": "dictionary"
    }
}

action = json.dumps(action_str)

In [15]:
bulk_data = ""

count = 0
for index, row in new_dict_df.iterrows():
    row_dict = {"item": row["dictionary"]}
    row_bulk = json.dumps(row_dict, ensure_ascii=False)
    bulk_data += f"{action}\n{row_bulk}\n"

In [16]:
with open(join(bulk_path, "dictionary.json"), "w", encoding="utf-8") as json_file:
    json_file.write(bulk_data)